## Proteomic and 3D Structural Evidence of Cysteine Oxidative PTMs.

Under oxidative stress Cysteines can undergo oxidative post-translational modifications (PTMs). 

In this study we compare the results of a proteomics study with observed oxidized forms of Cysteines in 3D structures from the Protein Data Bank (PDB).

* Proteomics Dataset

The study by Akter, et al. compares the differences between S-Sulfinylations (R-SO2H) and S-Sulfenylations (R-SOH) in A549 and HeLa cell lines.

Chemical proteomics reveals new targets of cysteine sulfinic acid reductase.
Akter S, Fu L, Jung Y, Conte ML, Lawson JR6, Lowther WT, Sun, Liu, Yang J, Carroll KS.
Nat Chem Biol. 2018 Sep 3. doi: [10.1038/s41589-018-0116-2](https://doi.org/10.1038/s41589-018-0116-2)

* PDB Dataset

BioJava-ModFinder: identification of protein modifications in 3D structures from the Protein Data Bank. Gao J, Prlić A,  Bi C  Bluhm WF, Dimitropoulos D, Xu D  Bourne, PE, Rose PW, Bioinformatics 2017, 33: 2047–2049. [doi: doi.org/10.1093/bioinformatics/btx101](https://doi.org/10.1093/bioinformatics/btx101)

In [1]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import asc, collect_set, collect_list, col, concat_ws, sort_array
from mmtfPyspark.datasets import pdbToUniProt, pdbPtmDataset
import pandas as pd
import numpy as np
from io import BytesIO
import xlrd
from ipywidgets import interact, IntSlider, widgets
import py3Dmol

In [2]:
# setup checkboxes for datasets
w1 = widgets.Checkbox(value=True, description='A549-RSO2H',disabled=False)
w2 = widgets.Checkbox(value=False, description='HeLa-RSO2H',disabled=False)
w3 = widgets.Checkbox(value=True, description='A549-RSOH',disabled=False)
w4 = widgets.Checkbox(value=False, description='HeLa-RSOH',disabled=False)

## Select one or more datasets (cell line-PTM)

In [3]:
display(w1, w2, w3, w4)

Checkbox(value=True, description='A549-RSO2H')

Checkbox(value=False, description='HeLa-RSO2H')

Checkbox(value=True, description='A549-RSOH')

Checkbox(value=False, description='HeLa-RSOH')

## Read and process datasets from supplementary materials

In [4]:
def read_datasets():
    dfs = []
    if w1.value:
        df1 = pd.read_excel('https://static-content.springer.com/esm/art%3A10.1038%2Fs41589-018-0116-2/MediaObjects/41589_2018_116_MOESM32_ESM.xlsx', sheet_name='A549', dtype=str)
        df1 = df1.assign(ptms=np.full((df1.shape[0], 1), "A549-RSO2H"))
        df1 = df1.rename(index=str, columns={"Modified site": "modifiedSite", "Uniprot Accession #": "uniprotAccession"})
        dfs.append(df1)

    if w2.value:
        df2 = pd.read_excel('https://static-content.springer.com/esm/art%3A10.1038%2Fs41589-018-0116-2/MediaObjects/41589_2018_116_MOESM32_ESM.xlsx', sheet_name='HeLa', dtype=str)
        df2 = df2.assign(ptms=np.full((df2.shape[0], 1), "HeLa-RSO2H"))
        df2 = df2.rename(index=str, columns={"Modified site": "modifiedSite", "Uniprot Accession #": "uniprotAccession"})
        dfs.append(df2)

    if w3.value:
        df3 = pd.read_excel('https://static-content.springer.com/esm/art%3A10.1038%2Fs41589-018-0116-2/MediaObjects/41589_2018_116_MOESM33_ESM.xlsx', sheet_name='A549', dtype=str)
        df3 = df3.assign(ptms=np.full((df3.shape[0], 1), "A549-RSOH"))
        df3 = df3.rename(index=str, columns={"Site #": "modifiedSite", "Uniprot Accession #": "uniprotAccession"})
        dfs.append(df3)

    if w4.value:
        df4 = pd.read_excel('https://static-content.springer.com/esm/art%3A10.1038%2Fs41589-018-0116-2/MediaObjects/41589_2018_116_MOESM33_ESM.xlsx', sheet_name='HeLa', dtype=str)
        df4 = df4.assign(ptms=np.full((df4.shape[0], 1), "HeLa-RSOH"))
        df4 = df4.rename(index=str, columns={"Site #": "modifiedSite", "Uniprot Accession #": "uniprotAccession"})
        dfs.append(df4)
        
    return dfs

In [5]:
# concatenate and process dataset
dfs = read_datasets()
df = pd.concat(dfs, ignore_index=True)
#display(df)
df = df[['ptms', 'modifiedSite', 'uniprotAccession', 'Description']]

df['modifiedSite'] = df['modifiedSite'].astype(np.int64)
pd.options.display.max_rows = None
display(df)

,ptms,modifiedSite,uniprotAccession,Description
0,A549-RSO2H,25,P63104,14-3-3 protein zeta/delta OS=Homo sapiens GN=Y...
1,A549-RSO2H,171,P52209,"6-phosphogluconate dehydrogenase, decarboxylat..."
2,A549-RSO2H,122,Q9H7C9,Mth938 domain-containing protein OS=Homo sapie...
3,A549-RSO2H,187,P00505,"Aspartate aminotransferase, mitochondrial OS=H..."
4,A549-RSO2H,477,P49748,Very long-chain specific acyl-CoA dehydrogenas...
5,A549-RSO2H,237,P49748,Very long-chain specific acyl-CoA dehydrogenas...
6,A549-RSO2H,733,Q9UKV3,Apoptotic chromatin condensation inducer in th...
7,A549-RSO2H,412,Q99424,Peroxisomal acyl-coenzyme A oxidase 2 OS=Homo ...
8,A549-RSO2H,286,Q562R1,Beta-actin-like protein 2 OS=Homo sapiens GN=A...
9,A549-RSO2H,45,O60218,Aldo-keto reductase family 1 member B10 OS=Hom...


## Map PTM locations to residues in PDB structures

In [6]:
# convert Pandas dataframe to a Spark dataframe
spark = SparkSession.builder.master("local[4]").appName("CysOxydationTo3DStructure").getOrCreate()
ds = spark.createDataFrame(df)
ds = ds.sort(ds.uniprotAccession, ds.modifiedSite)

Download PDB to UniProt mappings and filter out residues that were not observed in the 3D structure.

In [7]:
up = pdbToUniProt.get_cached_residue_mappings().filter("pdbResNum IS NOT NULL")

Joint PTM with PDB data if the UniProt Id and UniProt residue numbers match

In [8]:
st = up.join(ds, (up.uniprotId == ds.uniprotAccession) & (up.uniprotNum == ds.modifiedSite))

## Get PTMs present in 3D Structure of the PDB

We retrieve oxidated forms of L-cysteine in the PDB using the [PSI-MOD Ontology](https://www.ebi.ac.uk/ols/ontologies/mod)
* [MOD:00210](https://www.ebi.ac.uk/ols/ontologies/mod/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMOD_00210) - oxydation to L-cysteine sulfenic acid (RSOH)
* [MOD:00267](https://www.ebi.ac.uk/ols/ontologies/mod/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMOD_00267) - oxydation to L-cysteine sulfinic acid (RSO2H)
* [MOD:00460](https://www.ebi.ac.uk/ols/ontologies/mod/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMOD_00460) - oxydation to L-cysteine sulfonic acid (RSO3H)

In [9]:
pt = pdbPtmDataset.get_ptm_dataset()
pt = pt.filter("psimodId = 'MOD:00210' OR psimodId = 'MOD:00267' OR psimodId = 'MOD:00460'")
print("Total number of oxidized cysteines in PDB: ", pt.count())
pt.show(4000)

Total number of oxidized cysteines in PDB:  3700
+----------+---------+-------+---------+-------+----+----------------+--------------+
|pdbChainId|pdbResNum|residue| psimodId|residId|ccId|        category|modificationId|
+----------+---------+-------+---------+-------+----+----------------+--------------+
|    1ACD.A|      117|    CSD|MOD:00267| AA0262| CSD|modified residue|           119|
|    1ACD.A|      117|    CSD|MOD:00267| AA0262| CSD|modified residue|           120|
|    1BI5.A|      164|    CSD|MOD:00267| AA0262| CSD|modified residue|           119|
|    1BI5.A|      164|    CSD|MOD:00267| AA0262| CSD|modified residue|           120|
|    1BQ6.A|      164|    CSD|MOD:00267| AA0262| CSD|modified residue|           119|
|    1BQ6.A|      164|    CSD|MOD:00267| AA0262| CSD|modified residue|           120|
|    1C0T.A|      280|    CSD|MOD:00267| AA0262| CSD|modified residue|           119|
|    1C0T.A|      280|    CSD|MOD:00267| AA0262| CSD|modified residue|           120|
|    

In [10]:
pt = pt.withColumnRenamed("pdbResNum", "resNum") # avoid two columns with identical names
st = st.join(pt, (st.pdbResNum == pt.resNum) & (st.structureChainId == pt.pdbChainId))

## Aggregate PTM data on a per residue and per chain basis

In [11]:
# Aggregate data
st = st.groupBy("structureChainId","pdbResNum","uniprotAccession","uniprotNum","Description").agg(collect_list("ptms").alias("ptms"))
st = st.withColumn("ptms", concat_ws((","), col("ptms")))
st = st.groupBy("structureChainId","uniprotAccession","Description").agg(collect_list("ptms").alias("ptms"), collect_list("pdbResNum").alias("pdbResNum"),  collect_list("uniprotNum").alias("uniprotNum"))

Keep only a single structural representative

In [12]:
st = st.drop_duplicates(["uniprotAccession","uniprotNum"])

## Show Table with PDB mappings

PDB residue numbers do not always match UniProt residue numbers. The table below shows the mapping for each protein chain.

In [13]:
# convert Spark dataframe back to a Pandas dataframe
sp = st.toPandas()
pd.options.display.max_rows = None # show all rows
display(sp)

,structureChainId,uniprotAccession,Description,ptms,pdbResNum,uniprotNum
0,4Y1V.A,P09382,Galectin-1 OS=Homo sapiens GN=LGALS1 PE=1 SV=2,"[A549-RSOH, A549-RSOH, A549-RSO2H,A549-RSOH]","[16, 42, 60]","[17, 43, 61]"
1,4Y1X.A,P09382,Galectin-1 OS=Homo sapiens GN=LGALS1 PE=1 SV=2,"[A549-RSO2H,A549-RSOH, A549-RSOH]","[60, 42]","[61, 43]"
2,1W6N.A,P09382,Galectin-1 OS=Homo sapiens GN=LGALS1 PE=1 SV=2,[A549-RSOH],[1016],[17]
3,4Y1Z.A,P09382,Galectin-1 OS=Homo sapiens GN=LGALS1 PE=1 SV=2,"[A549-RSOH, A549-RSO2H,A549-RSOH, A549-RSOH]","[42, 60, 16]","[43, 61, 17]"
4,1GSN.A,P00390,"Glutathione reductase, mitochondrial OS=Homo s...","[A549-RSOH, A549-RSOH]","[63, 423]","[107, 467]"
5,5M35.A,P63104,14-3-3 protein zeta/delta OS=Homo sapiens GN=Y...,"[A549-RSO2H,A549-RSOH]",[25],[25]
6,1OEU.A,P18031,Tyrosine-protein phosphatase non-receptor type...,"[A549-RSO2H,A549-RSO2H]",[215],[215]
7,1PRX.B,P30041,Peroxiredoxin-6 OS=Homo sapiens GN=PRDX6 PE=1 ...,"[A549-RSO2H,A549-RSOH]",[47],[47]
8,2IBU.D,P24752,"Acetyl-CoA acetyltransferase, mitochondrial OS...","[A549-RSO2H,A549-RSOH]",[126],[126]
9,4Y1U.B,P09382,Galectin-1 OS=Homo sapiens GN=LGALS1 PE=1 SV=2,"[A549-RSOH, A549-RSO2H,A549-RSOH, A549-RSOH]","[16, 60, 42]","[17, 61, 43]"


In [14]:
def view_modifications(df, cutoff_distance, *args):

    def view3d(show_labels=True,show_bio_assembly=False, show_surface=False, i=0):
        pdb_id, chain_id = df.iloc[i]['structureChainId'].split('.')
        res_num = df.iloc[i]['pdbResNum']
        labels = df.iloc[i]['ptms']
        
        # print header
        print ("PDB Id: " + pdb_id + " chain Id: " + chain_id)
        
        # print any specified additional columns from the dataframe
        for a in args:
            print(a + ": " + df.iloc[i][a])
        
        mod_res = {'chain': chain_id, 'resi': res_num}  
        
        # select neigboring residues by distance
        surroundings = {'chain': chain_id, 'resi': res_num, 'byres': True, 'expand': cutoff_distance}
        
        viewer = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly})
    
        # polymer style
        viewer.setStyle({'cartoon': {'color': 'spectrum', 'width': 0.6, 'opacity':0.8}})
        # non-polymer style
        viewer.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
        
        # style for modifications
        viewer.addStyle(surroundings,{'stick':{'colorscheme':'orangeCarbon', 'radius': 0.15}})
        viewer.addStyle(mod_res, {'stick':{'colorscheme':'redCarbon', 'radius': 0.4}})
        viewer.addStyle(mod_res, {'sphere':{'colorscheme':'gray', 'opacity': 0.7}})
        
        # set residue labels    
        if show_labels:
            for residue, label in zip(res_num, labels):
                viewer.addLabel(residue + ": " + label, \
                                {'fontColor':'black', 'fontSize': 9, 'backgroundColor': 'lightgray'}, \
                                {'chain': chain_id, 'resi': residue})

        viewer.zoomTo(surroundings)
        
        if show_surface:
            viewer.addSurface(py3Dmol.SES,{'opacity':0.8,'color':'lightblue'})

        return viewer.show()
       
    s_widget = IntSlider(min=0, max=len(df)-1, description='Structure', continuous_update=False)
    
    return interact(view3d, show_labels=True, show_bio_assembly=False, show_surface=False, i=s_widget)

## Visualize Results
Residues with reported modifications are shown in an all atom prepresentation as red sticks with transparent spheres. Each modified residue position is labeled by the PDB residue number and the type of the modification. Residues surrounding modified residue (within 6 A) are highlighted as yellow sticks. Small molecules within the structure are rendered as gray sticks.

* Move slider to browse through the results
* To rotate the structure, hold down the left mouse button and move the mouse.

In [15]:
view_modifications(sp, 6, 'uniprotAccession', 'Description');

interactive(children=(Checkbox(value=True, description='show_labels'), Checkbox(value=False, description='show…

## Matching evidence was found for the following proteins

In [16]:
rs = sp[['uniprotAccession', 'Description']].drop_duplicates()
display(rs)

,uniprotAccession,Description
0,P09382,Galectin-1 OS=Homo sapiens GN=LGALS1 PE=1 SV=2
4,P00390,"Glutathione reductase, mitochondrial OS=Homo s..."
5,P63104,14-3-3 protein zeta/delta OS=Homo sapiens GN=Y...
6,P18031,Tyrosine-protein phosphatase non-receptor type...
7,P30041,Peroxiredoxin-6 OS=Homo sapiens GN=PRDX6 PE=1 ...
8,P24752,"Acetyl-CoA acetyltransferase, mitochondrial OS..."
11,P09211,Glutathione S-transferase P OS=Homo sapiens GN...
14,P31947,14-3-3 protein sigma OS=Homo sapiens GN=SFN PE...
15,P14618,Pyruvate kinase PKM OS=Homo sapiens GN=PKM PE=...


In [17]:
spark.stop()